ISTRUZIONI

- Upload file layer.py e resnet50kerv.py

- in resnet50kerv.py cambia num_class=200 in quello che serve a te

- cambio poooling in avg etc. vedi paper

In [1]:
import os
if not os.path.isdir('./imagenet'):
  !git clone https://github.com/alexmanni/imagenet.git

fatal: destination path 'imagenet' already exists and is not an empty directory.


In [2]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalizes tensor with mean and standard deviation
])
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])                                    
])
DATA_DIR =  '/content/imagenet/imagenet/tiny-imagenet-200/train'
# Prepare Pytorch train/test Datasets
train_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=train_transform)
test_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=eval_transform)

train_indexes = [idx for idx in range(len(train_dataset)) if idx % 2]
test_indexes = [idx for idx in range(len(test_dataset)) if not idx % 2]

train_dataset = Subset(train_dataset, train_indexes)
test_dataset = Subset(test_dataset, test_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))


DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 200 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 16    # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 15      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
import resnet50kerv as resnetk
net = resnetk.resnet50()

Train Dataset: 50000
Test Dataset: 50000


In [0]:


# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is mandatory to study torchvision.models.alexnet source code

# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  # Step the scheduler
  scheduler.step() 

Starting epoch 1/15, LR = [0.001]
Step 0, Loss 6.895986080169678
Step 10, Loss 6.977899551391602
Step 20, Loss 6.93619441986084
Step 30, Loss 6.864353656768799
Step 40, Loss 6.759836196899414
Step 50, Loss 6.770820140838623
Step 60, Loss 7.080810546875
Step 70, Loss 6.741914749145508
Step 80, Loss 7.066752910614014
Step 90, Loss 6.98439884185791
Step 100, Loss 7.053464889526367
Step 110, Loss 7.059270858764648
Step 120, Loss 7.052088737487793
Step 130, Loss 6.911147117614746
Step 140, Loss 6.895445823669434
Step 150, Loss 6.861253261566162
Step 160, Loss 6.986120223999023
Step 170, Loss 6.916736602783203
Step 180, Loss 6.686980247497559
Step 190, Loss 6.642154216766357
Step 200, Loss 6.793854236602783
Step 210, Loss 6.861050128936768
Step 220, Loss 6.890669345855713
Step 230, Loss 6.876396179199219
Step 240, Loss 6.859894275665283
Step 250, Loss 6.972813606262207
Step 260, Loss 6.867968559265137
Step 270, Loss 6.898643493652344
Step 280, Loss 6.764869213104248
Step 290, Loss 6.86786317

In [0]:
torch.save(net.state_dict(), '/content/pth')
torch.save(net, '/content/pth')
from tqdm import tqdm
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 25/25 [00:03<00:00,  8.85it/s]

Test Accuracy: 0.0975
